# Installing necesssary libraries

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# importing necessary libraries
import os
import time
import shutil
import numpy as np
import pandas as pd
from tqdm import tqdm
from matplotlib import pyplot as plt

from datasets import load_dataset, DatasetDict
from peft import LoraConfig, get_peft_model, TaskType

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import GenerationConfig, TrainingArguments, Trainer

In [ ]:
# health-Q&A dataset
huggingface_dataset_name = "RafaelMPereira/HealthCareMagic-100k-Chat-Format-en"

ds = load_dataset(huggingface_dataset_name, split='train')


README.md:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

HealthCareMagic-100k-en.jsonl:   0%|          | 0.00/125M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/112165 [00:00<?, ? examples/s]

In [ ]:
import re
# Function to split text into "human" and "bot" columns
def split_text(example):
    match = re.match(r"<human>:\s*(.*?)\s*<bot>:\s*(.*)", example["text"], re.DOTALL)
    if match:
        return {"human": match.group(1).strip(), "bot": match.group(2).strip()}
    return {"human": "", "bot": ""}  # Handle cases where the format is incorrect

# Apply the function to the dataset
ds = ds.map(split_text, remove_columns=["text"])
ds

Map:   0%|          | 0/112165 [00:00<?, ? examples/s]

Dataset({
    features: ['human', 'bot'],
    num_rows: 112165
})

In [ ]:
# Selecting only 12k samples for faster fine-tuning
ds = ds.select(range(12000))
ds

Dataset({
    features: ['human', 'bot'],
    num_rows: 12000
})

In [ ]:
# Split the dataset into train and test
train_test_split = ds.train_test_split(test_size=0.05, seed=42)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

train_validation_split = train_dataset.train_test_split(test_size=0.05, seed=42)
train_dataset = train_validation_split['train']
validation_dataset = train_validation_split['test']

print(ds)

Dataset({
    features: ['human', 'bot'],
    num_rows: 12000
})


In [ ]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

In [ ]:
my_device = "cuda" if torch.cuda.is_available() else "cpu"
print("My Device: {}".format(my_device))

My Device: cuda


In [ ]:
# initialize the model
model_name = "HuggingFaceTB/SmolLM2-360M" # HuggingFaceTB/SmolLM2-135M, HuggingFaceTB/SmolLM2-360M, HuggingFaceTB/SmolLM2-1.7B, HuggingFaceTB/SmolLM2-1.7B-Instruct
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16).to(my_device)

tokenizer_config.json:   0%|          | 0.00/3.66k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/831 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/724M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
tokenizer.pad_token = tokenizer.eos_token  # decoder-only models typically don't have a padding token pre-defined
model.config.pad_token_id = tokenizer.pad_token_id # padding is needed for batching

In [ ]:
# Chat template
def generate_chat_template(query_text, my_tokenizer, my_device, instruct_model=False):
    # create a system message

    if instruct_model:
        messages = [{"role": "user", "content": query_text}]
        input_text = my_tokenizer.apply_chat_template(messages, tokenize=False)
        inputs = my_tokenizer.encode(input_text, return_tensors="pt").to(my_device)
    else:
        inputs = my_tokenizer.encode(query_text, return_tensors="pt").to(my_device)

    return inputs

def generate_output(my_inputs, my_tokenizer, my_model, max_tokens = 50, temp = 0.3, top_p = 0.9, top_k=50, penalty_score=1.2, do_sample = True, instruct_model=False):

    if instruct_model:
        outputs = my_model.generate(my_inputs, max_new_tokens=max_tokens, temperature=temp, top_p=top_p, top_k=top_k, repetition_penalty=penalty_score, do_sample=do_sample)
        output_text = my_tokenizer.decode(outputs[0], skip_special_tokens=True)
        cleaned_output_text = output_text.split("<|im_start|>assistant")[1].split("<|im_end|>")[0].strip()
    else:
         outputs = my_model.generate(my_inputs, max_new_tokens=max_tokens, temperature=temp, top_p=top_p, top_k=top_k, repetition_penalty=penalty_score, do_sample=do_sample,
                                     eos_token_id=my_tokenizer.eos_token_id)
         cleaned_output_text = my_tokenizer.decode(outputs[0], skip_special_tokens=True)

    return cleaned_output_text

In [ ]:
def tune_generate_chat_template(query_text, my_tokenizer, my_device, instruct_model=False):
    """
    Formats the query based on the instruction tuning prompt template.
    """

    # Apply the updated instruction-tuned prompt template
    formatted_prompt = f"""
    ### Instruction:
    You are an AI medical assistant. Respond to the patient request, help them in curing the problem which they are facing by providing them proper diagnosis.

    ### Patient Request:
    {query_text}

    ### Response:
    """

    if instruct_model:
        messages = [{"role": "user", "content": formatted_prompt}]
        input_text = my_tokenizer.apply_chat_template(messages, tokenize=False)
        inputs = my_tokenizer.encode(input_text, return_tensors="pt").to(my_device)
    else:
        inputs = my_tokenizer.encode(formatted_prompt, return_tensors="pt").to(my_device)

    return inputs

def tune_generate_output(my_inputs, my_tokenizer, my_model, max_tokens=50, temp=0.3, top_p=0.9, top_k=50, penalty_score=1.2, do_sample=True, instruct_model=False):
    """
    Generates a response from the model based on the input.
    """

    outputs = my_model.generate(
        my_inputs,
        max_new_tokens=max_tokens,
        temperature=temp,
        top_p=top_p,
        top_k=top_k,
        repetition_penalty=penalty_score,
        do_sample=do_sample,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=my_tokenizer.eos_token_id
    )

    # Decode output and clean it up
    output_text = my_tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Ensure safe parsing without hardcoded token removal
    cleaned_output_text = output_text.strip()

    return cleaned_output_text

In [ ]:
sample_query = "Question: What is the capital of Germany?"
sample_inputs = generate_chat_template(sample_query, tokenizer, my_device)
sample_output = generate_output(sample_inputs, tokenizer, model, max_tokens=20, temp = 0.6, top_p = 0.6, top_k=50, penalty_score=1.2, do_sample = True, instruct_model=False)

print(sample_query)
print("="*10)
print(sample_output)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Question: What is the capital of Germany?
Question: What is the capital of Germany?
Answer: Berlin.

10) Which country has a population over 25 million people


In [ ]:
train_dataset[0]

{'human': 'While on a dialisys treatment I started to cramp in my toes and asked that the removal of fluid be stopped. After about 20 min. my calf muscle cramped up so quick and hard, I felt a sharp pop severe pain. I could not walk very far. Did I rupture or tear the upper part of the muscle.?',
 'bot': 'Hi, Leg cramps are a common and usually harmless condition where the muscles in your leg suddenly become tight and painful. It usually occurs in the calf muscles and in most cases there is no obvious cause of it If it happens sporadically then there is no need for any treatment or tests. But if this is common and affect your daily routine then some measures should be taken. In some cases, there may be predisposing conditions such can be seen in pregnancy, after exercise and certain types of medication, such as statins or in liver disease. The best way to manage this is to stretch your muscles regularly. In some cases, magnesium or calcium supplements may help Have enough fluids and av

##  Inference on the pretrained model


In [ ]:
sample_idx = 10

sample_query = train_dataset['human'][sample_idx]
sample_response = train_dataset['bot'][sample_idx]
sample_inputs = generate_chat_template(sample_query, tokenizer, my_device)
sample_output = generate_output(sample_inputs, tokenizer, model, max_tokens=50, temp = 0.7, top_p = 0.6, top_k=50, penalty_score=1.2, do_sample = True, instruct_model=False)

print("="*10)
print(sample_query)
print("="*10)
print(sample_response)
print("="*10)
print(sample_output)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


My boyfriend was recently diagnosed with Wolff-Parkinson-White Syndrome. I did some research, and discovered that he has nearly all of the symptoms of heart block and many of the symptoms of heart failure. Whenever he has an episode with his heart, he gets very dizzy, nauseous, has chest pain and cold sweats, and eventually has a blackout. Is there anything that I can do to alleviate his symptoms when they occur?
Hello, thanks for reaching out to us. I understand your worry towards your boyfriends' health. For treatment of patients with Wolff-Parkinson-White, it depends on lots of factors like presence of other diseases that might be present, the degree of severity etc. The best advice I have for you is to encourage him to see his cardiologist to know the best treatment possible in his case. I hope he gets better soon. Warm regards
My boyfriend was recently diagnosed with Wolff-Parkinson-White Syndrome. I did some research, and discovered that he has nearly all of the symptoms of heart

In [ ]:
print(print_number_of_trainable_model_parameters(model))

trainable model parameters: 361821120
all model parameters: 361821120
percentage of trainable model parameters: 100.00%


In [ ]:
model  # this will show us the model architecture

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(49152, 960)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=960, out_features=960, bias=False)
          (k_proj): Linear(in_features=960, out_features=320, bias=False)
          (v_proj): Linear(in_features=960, out_features=320, bias=False)
          (o_proj): Linear(in_features=960, out_features=960, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=960, out_features=2560, bias=False)
          (up_proj): Linear(in_features=960, out_features=2560, bias=False)
          (down_proj): Linear(in_features=2560, out_features=960, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((960,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((960,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((960,), eps=1e-05)
    (rotary_emb): LlamaRotaryEm

In [ ]:
## LORA ##
lora_config = LoraConfig(
    r=8, # Rank
    lora_alpha=8,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"], #"gate_proj", "up_proj", "down_proj"
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

peft_model = get_peft_model(model, lora_config).to(my_device)

print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 1638400
all model parameters: 363459520
percentage of trainable model parameters: 0.45%


**Just training 0.45% parameters of the total model**

In [ ]:
## TRY DIFFERENT PROMPTS ##
sample_idx = 0

sample_query = train_dataset['human'][sample_idx]


prompt = f"""
### Instruction:
You are an AI medical assistant. Respond to the patient request, help them in curing the problem which they are facing by providing them proper diagnosis.

### Patient Request:
{sample_query}

### Response:
"""

inputs = tokenizer.encode(prompt, return_tensors="pt").to(my_device)
outputs = model.generate(inputs, max_new_tokens=50, temperature=0.6, top_p=0.7, top_k=50, repetition_penalty=1.2, do_sample=True, eos_token_id=tokenizer.eos_token_id)
sample_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

sample_response = train_dataset['bot'][sample_idx]

print("="*10)
print(sample_query)
print("="*10)
print(sample_response)
print("="*10)
print(sample_output)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


While on a dialisys treatment I started to cramp in my toes and asked that the removal of fluid be stopped. After about 20 min. my calf muscle cramped up so quick and hard, I felt a sharp pop severe pain. I could not walk very far. Did I rupture or tear the upper part of the muscle.?
Hi, Leg cramps are a common and usually harmless condition where the muscles in your leg suddenly become tight and painful. It usually occurs in the calf muscles and in most cases there is no obvious cause of it If it happens sporadically then there is no need for any treatment or tests. But if this is common and affect your daily routine then some measures should be taken. In some cases, there may be predisposing conditions such can be seen in pregnancy, after exercise and certain types of medication, such as statins or in liver disease. The best way to manage this is to stretch your muscles regularly. In some cases, magnesium or calcium supplements may help Have enough fluids and avoid long sitting, regu

In [ ]:
sample_idx = 0

sample_query = train_dataset['human'][sample_idx]
sample_response = train_dataset['bot'][sample_idx]

sample_inputs = tune_generate_chat_template(sample_query, tokenizer, my_device)
sample_output = tune_generate_output(sample_inputs, tokenizer, model, max_tokens=100, temp = 0.7, top_p = 0.6, top_k=50, penalty_score=1.2, do_sample = True, instruct_model=False)


print("="*10)
print(sample_query)
print("="*10)
print(sample_response)
print("="*10)
print(sample_output)

While on a dialisys treatment I started to cramp in my toes and asked that the removal of fluid be stopped. After about 20 min. my calf muscle cramped up so quick and hard, I felt a sharp pop severe pain. I could not walk very far. Did I rupture or tear the upper part of the muscle.?
Hi, Leg cramps are a common and usually harmless condition where the muscles in your leg suddenly become tight and painful. It usually occurs in the calf muscles and in most cases there is no obvious cause of it If it happens sporadically then there is no need for any treatment or tests. But if this is common and affect your daily routine then some measures should be taken. In some cases, there may be predisposing conditions such can be seen in pregnancy, after exercise and certain types of medication, such as statins or in liver disease. The best way to manage this is to stretch your muscles regularly. In some cases, magnesium or calcium supplements may help Have enough fluids and avoid long sitting, regu

In [ ]:
def tokenize_function(examples):
    """
    Tokenizes the dataset using the fine-tuning instruction prompt format.
    Handles batched processing by iterating over each example.
    """

    # Construct the full prompt for each example in the batch
    formatted_prompts = [
        f"""
        ### Instruction:
        You are an AI medical assistant. Respond to the patient request, help them in curing the problem which they are facing by providing them proper diagnosis.

        ### Patient Request:
        {human}

        ### Response:
        {bot}
        """
        for human, bot in zip(examples["human"], examples["bot"])
    ]

    # Tokenize each formatted prompt
    tokenized_outputs = tokenizer(
        formatted_prompts,
        padding="max_length",
        truncation=True,
        max_length=512,  # Adjust as needed
    )
    # Return tokenized inputs
    return {
        "input_ids": tokenized_outputs["input_ids"],
        "attention_mask": tokenized_outputs["attention_mask"],
        "labels": tokenized_outputs["input_ids"],  # Causal LM: labels = input_ids
    }

# Tokenizing and processing all dataset splits
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_train_dataset = tokenized_train_dataset.remove_columns(['human', 'bot'])

tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = tokenized_test_dataset.remove_columns(['human', 'bot'])

tokenized_validation_dataset = validation_dataset.map(tokenize_function, batched=True)
tokenized_validation_dataset = tokenized_validation_dataset.remove_columns(['human', 'bot'])

Map:   0%|          | 0/10830 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Map:   0%|          | 0/570 [00:00<?, ? examples/s]

In [ ]:
## now merge the splits into a single dataset format
tokenized_datasets = DatasetDict({
    'train': tokenized_train_dataset,
    'validation': tokenized_validation_dataset,
    'test': tokenized_test_dataset
})

print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

print(tokenized_datasets)

Shapes of the datasets:
Training: (10830, 3)
Validation: (570, 3)
Test: (600, 3)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10830
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 570
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 600
    })
})


# Training the model

In [ ]:
#### PEFT Training ####
output_dir = f'/content/SmolLM2-fine-tuned/health_qa_base_tune_peft-350M-{str(time.strftime("%Y_%m_%d_%H_%M"))}'   # define the directory to store the results

## DO IT FALSE, IF THERE ARE ANY VALUABLE MODELS!!!!
if False:
  shutil.rmtree('tuned_models_SmolLM2')

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

step_size = 100 # we have 11k in total
lr = 1e-3
n_epochs = 2
batch_size = 8
gradient_acc = 4
warmup_steps = 50 # previously:0
peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=False,
    learning_rate=lr,
    num_train_epochs=n_epochs,
    logging_steps=step_size,  # Logs training loss every x.th steps
    save_steps=step_size,
    warmup_steps=warmup_steps,
    logging_strategy="steps",  # Ensures logs are printed every x.th steps
    evaluation_strategy="steps",  # Runs validation after each epoch
    save_strategy="steps",  # Saves model checkpoints after each epoch
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=gradient_acc,
    no_cuda=not torch.cuda.is_available(),
    report_to="none"  # Disable WandB
)

peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_datasets["train"],
   eval_dataset=tokenized_datasets['validation']
)

peft_trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss,Validation Loss
100,3.066200,1.427497
200,1.413800,1.395630


Step,Training Loss,Validation Loss
100,3.066200,1.427497
200,1.413800,1.395630
300,1.397300,1.380561
400,1.392000,1.370290
500,1.354700,1.363531
600,1.364900,1.359313


TrainOutput(global_step=676, training_loss=1.6326817168286567, metrics={'train_runtime': 9495.9248, 'train_samples_per_second': 2.281, 'train_steps_per_second': 0.071, 'total_flos': 2.10000002777088e+16, 'train_loss': 1.6326817168286567, 'epoch': 1.9955686853766617})

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
from peft import PeftModel

# Define the model path (same as output_dir)
model_path = "/content/SmolLM2-fine-tuned/health_qa_base_tune_peft-350M-2025_04_07_12_50/checkpoint-676/"  # Replace with the actual timestamp

# Load base model (Make sure it's the same base model used for fine-tuning)
base_model_name = "HuggingFaceTB/SmolLM2-360M"  # Replace with the actual base model name
base_model = AutoModelForCausalLM.from_pretrained(base_model_name)

# Load fine-tuned LoRA model
peft_model = PeftModel.from_pretrained(base_model, model_path)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

# Move model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
peft_model.to(device)

# Set model to evaluation mode
peft_model.eval()


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(49152, 960)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=960, out_features=960, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=960, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=960, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(
       

In [ ]:
# Inference on fine-tuned model
def generate_response(prompt, max_length=256):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        output = peft_model.generate(**inputs, max_length=max_length)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Example usage:
prompt = "What are the symptoms of depression?"
response = generate_response(prompt)
print(response)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


What are the symptoms of depression?

Depression is a serious mental health condition that can affect your mood, energy, sleep, appetite, concentration, and self-esteem. It can also affect your relationships and ability to work.

What are the signs of depression?

The signs of depression can include:

  • Feeling sad, hopeless, or empty
  • Feeling worthless or guilty
  • Feeling restless or irritable
  • Having trouble sleeping or sleeping too much
  • Having trouble concentrating
  • Having thoughts of death or suicide

What are the causes of depression?

There are many causes of depression. Some of the most common causes include:

  • Genetics: Depression is more common in people who have a family history of depression.
  • Brain chemistry: Depression can be caused by changes in the brain’s chemistry.
  • Stress: Stress can trigger depression.
  • Life events: Life events such as the loss of a loved one or the death of a loved one can trigger depression.
  • Medical conditions: Depr

In [ ]:
from peft import PeftModel
from transformers import AutoModelForCausalLM

# Merge LoRA weights into base model
merged_model = peft_model.merge_and_unload()

# Save & push merged full model
merged_model.push_to_hub("Kaith-jeet123/SmolLM2-fine-tuned-healthQA")
tokenizer.push_to_hub("Kaith-jeet123/SmolLM2-fine-tuned-healthQA")


model.safetensors:   0%|          | 0.00/1.45G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Kaith-jeet123/SmolLM2-fine-tuned-healthQA/commit/070af337f234b1c0f2310ae8a56320b4fc5ae31d', commit_message='Upload tokenizer', commit_description='', oid='070af337f234b1c0f2310ae8a56320b4fc5ae31d', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Kaith-jeet123/SmolLM2-fine-tuned-healthQA', endpoint='https://huggingface.co', repo_type='model', repo_id='Kaith-jeet123/SmolLM2-fine-tuned-healthQA'), pr_revision=None, pr_num=None)